In [30]:
import pandas as pd
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn_pandas import DataFrameMapper

In [2]:
#Function for extracting wh-word feature from the question
def wh_word(sent):
    whPattern = re.compile(r'(who|what|when|^is|^are|^was|^were)', re.IGNORECASE)
    whMatch = whPattern.search(sent)
    if whMatch:
        whWord = whMatch.group()
        if whWord in ['is','Is','are','Are','was','Was','Were','were']:
            return 'affirmation'
        else:
            return str(whWord).lower()
    else:
        return 'none'

In [91]:
#Function for loading data from text file and storing in a pandas dataframe along with the wh_word feature
def load_file1(f):
    f = open(f,'r')
    res = []
    for line in f:
        label, question = line.split(" ", 1)
        res.append((label, question))
    df = pd.DataFrame(res, columns= ['label','question'])
    LE = LabelEncoder()
    df['whword'] = LE1.fit_transform(map(wh_word,df['question']))
    return df

In [92]:
#Function for loading data from file downloaded from UIUC website and storing in a pandas dataframe along with the wh_word feature
def load_file(f):
    f = open(f,'r')
    res = []
    for line in f:
        label, question = line.split(" ", 1)
        #"""
        if re.compile(r'date').search(label):
            label = 'when'
        elif re.compile(r'HUM').search(label):
            label = 'who'
        elif re.compile(r'ABBR|ENTY|DESC').search(label):
            label = 'what'
        else:
            label = 'Unknown'
        #"""
        res.append((label, question))
    df = pd.DataFrame(res, columns= ['label','question'])
    LE = LabelEncoder()
    df['whword'] = LE1.fit_transform(map(wh_word,df['question']))
    return df

In [86]:
df = load_file('train_1000.label') #for loading data from UIUC website
#df = load_file1('filename') #for loading data from a text file
msk = np.random.rand(len(df)) < 0.8

train = df[msk]
test = df[~msk]

mapper = DataFrameMapper([
    ('whword', None),
    ('question',CountVectorizer(decode_error=u'ignore', ngram_range=(1, 2), analyzer=u'word', max_df=0.8, min_df=0.002))
])

X_train = mapper.fit_transform(train)
X_test = mapper.transform(test)

LE2 = LabelEncoder()
y_train = LE2.fit_transform(train['label'])
y_test = LE2.transform(test['label'])

In [111]:
clf = svm.LinearSVC(C = 0.9).fit(X_train,y_train)

In [112]:
print clf

LinearSVC(C=0.9, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)


In [113]:
predict = clf.predict(X_test)

In [114]:
accuracy_score(y_test, predict)

0.86387434554973819